In [ ]:
!pip install ultralytics --quiet
from ultralytics import YOLO
import torch
from torchvision import transforms
from PIL import Image
import os
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
COMBINED_MODEL_PATH = "/content/drive/MyDrive/yolov8-road-detection/experiment/combined-1k/weights/best.pt"
DAY_MODEL_PATH = "/content/drive/MyDrive/yolov8-road-detection/experiment/day/weights/best.pt"
NIGHT_MODEL_PATH = "/content/drive/MyDrive/yolov8-road-detection/experiment/night/weights/best.pt"
CLASSIFIER_PATH = "/content/drive/MyDrive/yolov8-road-detection/experiment/classifier.pt"

TEST_IMAGES_DIR = "/content/drive/MyDrive/yolov8-road-detection/experiment/test/images"
TEST_LABELS_DIR = "/content/drive/MyDrive/yolov8-road-detection/experiment/test/labels"

ROUTED_DAY_DIR = "/content/routed/day"
ROUTED_NIGHT_DIR = "/content/routed/night"

os.makedirs(ROUTED_DAY_DIR, exist_ok=True)
os.makedirs(ROUTED_NIGHT_DIR, exist_ok=True)

In [ ]:
combined_model = YOLO(COMBINED_MODEL_PATH)
combined_results = combined_model.val(
    data="/content/drive/MyDrive/yolov8-road-detection/experiment/test/test.yaml",
    save=False,
    plots=False,
    split='val'
)
combined_map50 = combined_results.box.map50
print(f"Combined Model mAP50: {combined_map50:.4f}")

In [ ]:
combined_small = "/content/drive/MyDrive/yolov8-road-detection/experiment/combined-500/weights/best.pt"
combined_small_model = YOLO(combined_small)
combined_small_results = combined_small_model.val(
    data="/content/drive/MyDrive/yolov8-road-detection/experiment/test/test.yaml",
    save=False,
    plots=False,
    split='val'
)
combined_small_map50 = combined_results.box.map50
print(f"Combined Small Model mAP50: {combined_small_map50:.4f}")

In [ ]:
DAY_IMAGES_DIR = "/content/drive/MyDrive/yolov8-road-detection/experiment/test-categorized/day"
NIGHT_IMAGES_DIR = "/content/drive/MyDrive/yolov8-road-detection/experiment/test-categorized/night"

day_set_model = YOLO(DAY_MODEL_PATH)
day_set_results = day_set_model.val(
    data="/content/drive/MyDrive/yolov8-road-detection/experiment/test-categorized/day/day.yaml",
    save=False,
    plots=False,
    split='val'
)
day_set_map50 = day_set_results.box.map50

night_set_model = YOLO(NIGHT_MODEL_PATH)
night_set_results = night_set_model.val(
    data="/content/drive/MyDrive/yolov8-road-detection/experiment/test-categorized/night/night.yaml",
    save=False,
    plots=False,
    split='val'
)
night_set_map50 = night_set_results.box.map50

modular_set_map50 = (day_set_map50 * 75 + night_set_map50 * 75) / 150
print(f"Modular Model mAP50: {modular_set_map50:.4f}")

In [ ]:
import os, shutil, yaml, contextlib, io
from ultralytics import YOLO
from tqdm import tqdm

ROUTED_DAY_DIR = "/content/routed/day/images"
ROUTED_NIGHT_DIR = "/content/routed/night/images"
ROUTED_DAY_LABELS = "/content/routed/day/labels"
ROUTED_NIGHT_LABELS = "/content/routed/night/labels"

for d in [ROUTED_DAY_DIR, ROUTED_NIGHT_DIR, ROUTED_DAY_LABELS, ROUTED_NIGHT_LABELS]:
    os.makedirs(d, exist_ok=True)

classifier = YOLO(CLASSIFIER_PATH)

for fname in tqdm(os.listdir(TEST_IMAGES_DIR)):
    if not fname.lower().endswith(('.jpg', '.png', '.jpeg')):
        continue

    img_path = os.path.join(TEST_IMAGES_DIR, fname)
    label_path = os.path.join(TEST_LABELS_DIR, fname.rsplit('.', 1)[0] + '.txt')

    with contextlib.redirect_stdout(io.StringIO()):
        result = classifier(img_path)
    cls = int(result[0].probs.top1)

    if cls == 0:
        shutil.copy(img_path, os.path.join(ROUTED_DAY_DIR, fname))
        shutil.copy(label_path, os.path.join(ROUTED_DAY_LABELS, fname.rsplit('.', 1)[0] + '.txt'))
    else:
        shutil.copy(img_path, os.path.join(ROUTED_NIGHT_DIR, fname))
        shutil.copy(label_path, os.path.join(ROUTED_NIGHT_LABELS, fname.rsplit('.', 1)[0] + '.txt'))

def write_yaml(val_dir, yaml_path):
    yaml_dict = {
        'train': 'not_used',
        'val': val_dir,
        'names': ['0','1','2'],
    }
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_dict, f)

DAY_YAML_PATH = "/content/routed/day_dataset.yaml"
NIGHT_YAML_PATH = "/content/routed/night_dataset.yaml"
write_yaml("/content/routed/day", DAY_YAML_PATH)
write_yaml("/content/routed/night", NIGHT_YAML_PATH)

day_model = YOLO(DAY_MODEL_PATH)
day_results = day_model.val(data=DAY_YAML_PATH, split='val', save=False, plots=False)
day_map50 = day_results.box.map50

night_model = YOLO(NIGHT_MODEL_PATH)
night_results = night_model.val(data=NIGHT_YAML_PATH, split='val', save=False, plots=False)
night_map50 = night_results.box.map50

num_day = len(os.listdir(ROUTED_DAY_DIR))
num_night = len(os.listdir(ROUTED_NIGHT_DIR))
total = num_day + num_night

modular_map50 = (day_map50 * num_day + night_map50 * num_night) / total

In [ ]:
import os
from ultralytics import YOLO
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm

# === Load classifier ===
classifier = YOLO('/content/drive/MyDrive/yolov8-road-detection/experiment/classifier.pt')

# === Base test folder ===
base_dir = '/content/drive/MyDrive/yolov8-road-detection/experiment/test-categorized'

# === Init
y_true = []
y_pred = []

label_map = {'day': 0, 'night': 1}

# === Loop over true labels
for label_name in ['day', 'night']:
    image_dir = os.path.join(base_dir, label_name, 'images')  # go into /day/images, /night/images

    if not os.path.exists(image_dir):
        continue  # skip if folder missing

    for fname in tqdm(os.listdir(image_dir), desc=f'Processing {label_name}'):
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img_path = os.path.join(image_dir, fname)

        # Inference
        result = classifier(img_path)
        pred_label = int(result[0].probs.top1)

        y_true.append(label_map[label_name])
        y_pred.append(pred_label)

# === Check if data was collected
if len(y_true) == 0 or len(y_pred) == 0:
    print("No valid predictions collected. Please check image paths and classifier output.")
else:
    print("Classification Report:")
    print(classification_report(
        y_true, y_pred,
        labels=[0, 1],
        target_names=['day', 'night'],
        zero_division=0
    ))
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred, labels=[0, 1]))

In [ ]:
combined_mp = combined_results.box.mp
combined_mr = combined_results.box.mr
combined_map50 = combined_results.box.map50
combined_map5095 = combined_results.box.map

day_mp = day_set_results.box.mp
day_mr = day_set_results.box.mr
day_map50 = day_set_results.box.map50
day_map5095 = day_set_results.box.map

night_mp = night_set_results.box.mp
night_mr = night_set_results.box.mr
night_map50 = night_set_results.box.map50
night_map5095 = night_set_results.box.map

combined_small_mp = combined_small_results.box.mp
combined_small_mr = combined_small_results.box.mr
combined_small_map50 = combined_small_results.box.map50
combined_small_map5095 = combined_small_results.box.map

modular_non_cls_mp = (day_mp * 75 + night_mp * 75) / 150
modular_non_cls_mr = (day_mr * 75 + night_mr * 75) / 150
modular_non_cls_map50 = (day_map50 * 75 + night_map50 * 75) / 150
modular_non_cls_map5095 = (day_map5095 * 75 + night_map5095 * 75) / 150

num_day = len(os.listdir(ROUTED_DAY_DIR))
num_night = len(os.listdir(ROUTED_NIGHT_DIR))
total = num_day + num_night

modular_cls_mp = (day_mp * num_day + night_mp * num_night) / total
modular_cls_mr = (day_mr * num_day + night_mr * num_night) / total
modular_cls_map50 = (day_map50 * num_day + night_map50 * num_night) / total
modular_cls_map5095 = (day_map5095 * num_day + night_map5095 * num_night) / total

print("Detection Evaluation Results")
print("-------------------------------------------------------------------------")
print(f"{'Model':<28} | {'Prec':<9} | {'Recall':<9} | {'mAP@0.5':<10} | {'mAP@0.5:0.95'}")
print("-------------------------------------------------------------------------")
print(f"{'Combined Model 1,000':<28} | {combined_mp:.4f}   | {combined_mr:.4f}   | {combined_map50:.4f}   | {combined_map5095:.4f}")
print(f"{'Combined Model 500':<28} | {combined_small_mp:.4f}   | {combined_small_mr:.4f}   | {combined_small_map50:.4f}   | {combined_small_map5095:.4f}")
print(f"{'Modular (Non-Classifier)':<28} | {modular_non_cls_mp:.4f}   | {modular_non_cls_mr:.4f}   | {modular_non_cls_map50:.4f}   | {modular_non_cls_map5095:.4f}")
print(f"{'Modular (With Classifier)':<28} | {modular_cls_mp:.4f}   | {modular_cls_mr:.4f}   | {modular_cls_map50:.4f}   | {modular_cls_map5095:.4f}")
print("-------------------------------------------------------------------------")